# 🧪 Proyecto Didáctico RAG: Barman Virtual con PDF

Este proyecto demuestra una arquitectura sencilla de **RAG (Retrieval-Augmented Generation)** aplicada a un escenario práctico y entretenido: un **asistente virtual para recetas de cócteles**, que responde preguntas en lenguaje natural utilizando como única fuente un documento PDF con recetas.

La solución está construida con Python y librerías open-source como LangChain, FAISS, HuggingFace y Transformers. El flujo completo incluye:

- 🧾 Extracción y limpieza de texto desde un PDF.
- ✂️ Divisiones en fragmentos (chunks) para contextualizar.
- 📚 Creación de un buscador semántico (retriever) con embeddings.
- 🧠 Generación de respuestas contextualizadas con el modelo **Flan-T5**.

Es un proyecto pensado **para aprendizaje y exploración técnica**, ideal como punto de partida para construir sistemas más complejos o adaptarlo a otros dominios (leyes, medicina, educación, etc.).

> ⚙️ Todo se ejecuta **en local**, sin depender de APIs de pago, lo que permite experimentar sin restricciones ni costes adicionales.


### 🧱 Función `safe_write` para guardar módulos sin sobrescribir

definimos la función `safe_write`, que permite crear archivos `.py` evitando sobrescribir archivos existentes sin confirmación. Será utilizada más adelante para generar los archivos de los módulos (`loader.py`, `retriever.py`, `generator.py`, etc.) de forma segura.


In [1]:

import os

def safe_write(filepath, content):
    if os.path.exists(filepath):
        res = input(f"⚠️ '{filepath}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
        if res != "s":
            print(f"⏩ '{filepath}' se ha mantenido sin cambios.")
            return
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"✅ Archivo '{{filepath}}' creado con éxito.")


### 🧪 Instalamos las dependencias necesarias para ejecutar el sistema de RAG

En esta celda instalamos todas las librerías que usaremos en nuestro flujo de recuperación y generación de respuestas. Incluimos:

- **LangChain y LangChain Community**: para la orquestación de cargado, embebido y recuperación de contexto.
- **Sentence Transformers**: para crear los embeddings semánticos con Hugging Face.
- **Transformers + Torch**: para ejecutar el modelo generador Flan-T5.
- **FAISS CPU**: como motor de búsqueda vectorial para los chunks del documento.
- **PyMuPDF y PyPDF**: para leer el PDF del recetario desde el disco.


In [2]:
# Instalamos solo las dependencias necesarias y relevantes para el flujo funcional del proyecto
!pip install -q langchain langchain-community sentence-transformers pymupdf pypdf faiss-cpu transformers torch accelerate



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

### 📥 Descargamos el PDF del recetario y extraemos su contenido limpio

En esta celda automatizamos la descarga del documento base: el recetario de cócteles llamado **"Guía del Barman"**.

1. Verificamos si el archivo ya existe. Si es así, preguntamos si se desea reemplazarlo.
2. Si no existe, lo descargamos directamente desde Google Drive usando `gdown`.
3. Abrimos el archivo PDF con `PyMuPDF` (`fitz`) y extraemos su texto página por página.
4. Realizamos una limpieza básica del texto (eliminando espacios y líneas vacías).
5. Finalmente, guardamos el texto limpio en un archivo `recetas.txt` dentro de la carpeta `data/`.

Este paso permite tener una copia legible y estructurada del documento para inspección rápida o debug si algo falla más adelante.


In [3]:
import os
import gdown
import fitz  # PyMuPDF

# 🔄 Asegurar que existe la carpeta para datos y modulos
# 🔧 Crear carpeta modules si no existe
os.makedirs("data", exist_ok=True)
os.makedirs("modules", exist_ok=True)

# 📦 ID del archivo en Google Drive
file_id = "1aFwI0tw6fClegyufHJcbHzKGmaMZq-Lz"
pdf_path = "data/guia_del_barman.pdf"

# 📥 Descargar el PDF con gdown
if os.path.exists(pdf_path):
    respuesta = input(f"⚠️ El archivo {pdf_path} ya existe. ¿Deseas reemplazarlo? (s/n): ").strip().lower()
    if respuesta == 's':
        gdown.download(id=file_id, output=pdf_path, quiet=False)
        print("✅ PDF reemplazado con éxito.")
    else:
        print("⏩ Se ha mantenido el archivo existente sin cambios.")
else:
    print("🔄 Descargando PDF desde Google Drive...")
    gdown.download(id=file_id, output=pdf_path, quiet=False)
    print("✅ PDF descargado a", pdf_path)

# 📄 Abrir el PDF con PyMuPDF y extraer texto completo
print("📖 Extrayendo texto del PDF...")
doc = fitz.open(pdf_path)
texto = ""
for page in doc:
    texto += page.get_text()
doc.close()
print("✅ Extracción completada.")

# 🧹 Limpieza mínima: eliminar líneas vacías y espacios
texto_limpio = "\n".join([line.strip() for line in texto.splitlines() if line.strip()])

# 💾 Guardar en data/recetas.txt
recetas_txt = "data/recetas.txt"
with open(recetas_txt, "w", encoding="utf-8") as f:
    f.write(texto_limpio)
print(f"📄 Texto limpio guardado en '{recetas_txt}'")



🔄 Descargando PDF desde Google Drive...


Downloading...
From: https://drive.google.com/uc?id=1aFwI0tw6fClegyufHJcbHzKGmaMZq-Lz
To: /content/data/guia_del_barman.pdf
100%|██████████| 274k/274k [00:00<00:00, 82.6MB/s]

✅ PDF descargado a data/guia_del_barman.pdf
📖 Extrayendo texto del PDF...
✅ Extracción completada.
📄 Texto limpio guardado en 'data/recetas.txt'


### ✂️ Dividimos el texto en fragmentos manejables (chunks)

En esta celda cargamos el texto limpio previamente extraído del PDF y lo dividimos en fragmentos más pequeños (chunks) para facilitar su uso posterior por el modelo.

1. Cargamos el archivo `recetas.txt` desde la carpeta `data/`.
2. Utilizamos `CharacterTextSplitter` de LangChain para dividir el texto por saltos de línea, con un tamaño máximo de 1000 caracteres por fragmento y una superposición de 100 caracteres para mantener el contexto entre bloques.
3. Finalmente, mostramos cuántos chunks se generaron y un ejemplo de contenido para verificar el resultado.

Este paso es clave para preparar el texto antes de crear embeddings o alimentar el modelo.


In [4]:
from langchain.text_splitter import CharacterTextSplitter

# 📥 Cargar el texto desde el archivo plano
with open("data/recetas.txt", "r", encoding="utf-8") as f:
    texto_limpio = f.read()

# ✂️ Inicializar el splitter
text_splitter = CharacterTextSplitter(
    separator="\n",       # Separador por saltos de línea
    chunk_size=1000,      # Tamaño máximo por chunk (caracteres)
    chunk_overlap=100,    # Superposición para contexto
)

# 🔄 Crear los chunks
chunks = text_splitter.split_text(texto_limpio)

# 📊 Mostrar resumen
print(f"✅ Texto dividido en {len(chunks)} chunks.")
print("🧪 Ejemplo de chunk:\n")
print(chunks[0][:500])  # Mostrar solo parte del primer chunk


✅ Texto dividido en 138 chunks.
🧪 Ejemplo de chunk:

Brenda
LA GUIA
DEL
BARMAN
Aperitivos
Según el diccionario aperitivo es todo lo que despierta el apetito. Y eso es lo
que  logran  ciertas  bebidas  que,  tomadas  antes  de  la  comida,  significan  una
especie de rito. Sobre todo en muchos países europeos, que , entre otras cosas,
también  producen  los  mejores  vinos  aperitivos.  Los  más  conocidos  son  el
jerez, el madeira y el vermounth.
Jerez
Esta  es  una  bebida  típicamente  española.  Nació  en  Andalucía,  en  la  antigua
ciudad de


### 🛠️ Añadimos la carpeta `modules/` al entorno de trabajo

En esta celda, nos aseguramos de que la carpeta `modules/`, donde están definidos nuestros scripts auxiliares (`loader.py`, `retriever.py`, `generator.py`, etc.), esté incluida en el path del sistema (`sys.path`). Esto es necesario para poder importarlos correctamente desde cualquier parte del notebook o del proyecto.

Este paso permite mantener una estructura modular del código sin tener problemas de importación.


In [5]:
import sys
import os

# 📁 Asegura que la carpeta 'modules' esté en el path
module_path = os.path.abspath("modules")
if module_path not in sys.path:
    sys.path.append(module_path)
    print("📦 'modules' agregado al sys.path")


📦 'modules' agregado al sys.path


### 🔧 Generamos el archivo `embedder.py` para cargar embeddings

En esta celda definimos el módulo `embedder.py`, el cual encapsula la carga del modelo de embeddings `all-MiniLM-L6-v2` desde HuggingFace. Este modelo transforma fragmentos de texto en vectores numéricos que luego son usados por FAISS para la búsqueda semántica.

Este script se guarda dentro de la carpeta `modules/`, y se utiliza en la creación del `retriever`.


In [6]:
import os

file_path = "modules/embedder.py"

if os.path.exists(file_path):
    res = input(f"⚠️ '{file_path}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
    if res != "s":
        print(f"⏩ '{file_path}' se ha mantenido sin cambios.")
    else:
        os.remove(file_path)

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write('''"""
embedder.py 📎
Módulo encargado de cargar el modelo de embeddings 'all-MiniLM-L6-v2'
usando HuggingFaceEmbeddings para transformar texto en vectores numéricos.
"""

from langchain_community.embeddings import HuggingFaceEmbeddings

def load_embedder(model_name: str = "all-MiniLM-L6-v2") -> HuggingFaceEmbeddings:
    """
    Carga el modelo HuggingFaceEmbeddings con el nombre especificado.

    Args:
        model_name (str): Nombre del modelo (por defecto: "all-MiniLM-L6-v2")

    Returns:
        HuggingFaceEmbeddings: Objeto cargado de embeddings
    """
    return HuggingFaceEmbeddings(model_name=model_name)
''')
    print(f"✅ Archivo '{file_path}' creado con éxito.")


✅ Archivo 'modules/embedder.py' creado con éxito.


### 🧠 Crear o cargar el sistema de recuperación de contexto (retriever)

En esta celda generamos o cargamos un `retriever`, que es el sistema responsable de encontrar fragmentos relevantes del documento para responder preguntas. Utilizamos FAISS junto con embeddings del modelo `all-MiniLM-L6-v2`.

- Si ya existe un índice guardado en disco (`retriever.pkl`), simplemente lo cargamos.
- Si no existe, se debe proporcionar una lista de documentos (`docs`) para construirlo desde cero.




In [7]:
import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

file_path = "modules/retriever.py"

# Verificar si existe y preguntar si se sobrescribe
if os.path.exists(file_path):
    res = input(f"⚠️ El archivo '{file_path}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
    if res != "s":
        print(f"⏩ '{file_path}' se ha mantenido sin cambios.")
    else:
        os.remove(file_path)

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write('''"""
retriever.py
Este módulo crea o carga un retriever local usando FAISS y embeddings HuggingFace.
"""

import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

def create_or_load_retriever(docs=None, save_path="retriever/faiss_index"):
    """
    Crea o carga el retriever desde disco. Si no existe, se requiere pasar los documentos.
    """
    retriever_file = os.path.join(save_path, "retriever.pkl")

    if os.path.exists(retriever_file):
        print("📦 Cargando retriever desde disco...")
        with open(retriever_file, "rb") as f:
            retriever = pickle.load(f)
    else:
        if not docs:
            raise ValueError("❌ No se encontraron datos para construir el retriever.")
        print("🧠 Creando nuevo retriever con embeddings...")
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        retriever = FAISS.from_documents(docs, embedding_model)
        os.makedirs(save_path, exist_ok=True)
        with open(retriever_file, "wb") as f:
            pickle.dump(retriever, f)

    return retriever.as_retriever()
''')
    print(f"✅ Archivo '{file_path}' creado o actualizado con éxito.")



✅ Archivo 'modules/retriever.py' creado o actualizado con éxito.


### 🤖 Generamos el archivo `generator.py` para respuestas contextuales

En esta celda creamos el archivo `generator.py`, encargado de usar un modelo de lenguaje (Flan-T5) para responder preguntas del usuario. Este modelo se alimenta del contexto relevante extraído del recetario usando un retriever semántico (FAISS).

El prompt guía al modelo para actuar como un barman experto, limitando sus respuestas únicamente a la información contenida en los documentos. Se utiliza `transformers` para el modelado, y el resultado es una respuesta natural y coherente basada en el texto disponible.


In [8]:
import os

file_path = "modules/generator.py"

# Verificar si existe y preguntar si se sobrescribe
if os.path.exists(file_path):
    res = input(f"⚠️ El archivo '{file_path}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
    if res != "s":
        print(f"⏩ '{file_path}' se ha mantenido sin cambios.")
    else:
        os.remove(file_path)

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write('''"""
generator.py
Genera respuestas con contexto usando un modelo de lenguaje (Flan-T5) y un retriever basado en FAISS.
"""

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain_core.retrievers import BaseRetriever

# Cargar modelo y tokenizer solo una vez
model_name = "google/flan-t5-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device set to use {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


def ask_with_context(query: str, retriever: BaseRetriever, max_context_length: int = 1500) -> str:
    """
    Genera una respuesta a partir de una pregunta y un retriever que devuelve documentos relevantes.
    """

    # Recuperar documentos contextuales
    context_docs = retriever.get_relevant_documents(query)
    context = "\\n\\n".join(doc.page_content for doc in context_docs)

    # Recortar contexto si es muy largo para Flan-T5
    if len(context) > max_context_length:
        context = context[:max_context_length]

    # Prompt
    prompt = f\"\"\"Eres un barman experto. Basado únicamente en el siguiente texto del recetario, responde la pregunta del usuario de forma clara y directa. Si no encuentras la información, simplemente di "No lo sé".

Recetario:
{context}

Pregunta: {query}
Respuesta:\"\"\"

    # Tokenizar y generar
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = model.generate(**inputs, max_new_tokens=200)
    respuesta = tokenizer.decode(output[0], skip_special_tokens=True)

    return respuesta.strip()
''')
    print(f"✅ Archivo '{file_path}' creado o actualizado con éxito.")




✅ Archivo 'modules/generator.py' creado o actualizado con éxito.


### 📄 Creamos el archivo `loader.py` para cargar y dividir el PDF

En esta celda generamos el módulo `loader.py`, que se encarga de leer el archivo PDF `guia_del_barman.pdf` y dividir su contenido en fragmentos más pequeños llamados *chunks*.

Utilizamos `PyPDFLoader` para extraer las páginas del PDF y luego aplicamos `RecursiveCharacterTextSplitter`, que divide el contenido respetando la estructura del texto, ideal para modelos que necesitan contexto fragmentado. Esta función será utilizada posteriormente para alimentar el sistema de recuperación semántica.


In [9]:
import os

file_path = "modules/loader.py"

# Verificar si existe y preguntar si se sobrescribe
if os.path.exists(file_path):
    res = input(f"⚠️ El archivo '{file_path}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
    if res != "s":
        print(f"⏩ '{file_path}' se ha mantenido sin cambios.")
    else:
        os.remove(file_path)

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write('''"""
loader.py
Carga documentos PDF y los divide en chunks usando LangChain.
"""

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_documents_and_chunk(ruta_pdf="data/guia_del_barman.pdf"):
    loader = PyPDFLoader(ruta_pdf)
    pages = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150
    )
    chunks = splitter.split_documents(pages)

    return chunks
''')
    print(f"✅ Archivo '{file_path}' creado o actualizado con éxito.")


✅ Archivo 'modules/loader.py' creado o actualizado con éxito.


### 🤖 Creamos el archivo `main.py` para ejecutar el sistema de preguntas con contexto automáticamente

Este script `main.py` es el **archivo orquestador del proyecto**, responsable de coordinar todas las piezas del sistema: carga del documento, creación del *retriever* (sistema de recuperación semántica) y generación de la respuesta por parte del modelo de lenguaje.

A diferencia de una ejecución con `input()` manual, esta versión está diseñada para **ejecución automática en Google Colab**, utilizando una **pregunta predefinida** como prueba funcional. Esto permite validar rápidamente si el sistema completo está funcionando sin intervención del usuario.

Los pasos son:
1. Se carga y divide el PDF en chunks.
2. Se crea o recupera el vector store (FAISS) con embeddings.
3. Se lanza una pregunta fija: *“¿Qué cóctel puedo hacer con lima y ron?”*.
4. Se imprime la respuesta generada por el modelo Flan-T5 con contexto.


In [10]:
import os

file_path = "modules/main.py"

# Verificar si existe y preguntar si se sobrescribe
if os.path.exists(file_path):
    res = input(f"⚠️ El archivo '{file_path}' ya existe. ¿Deseas reescribirlo? (s/n): ").strip().lower()
    if res != "s":
        print(f"⏩ '{file_path}' se ha mantenido sin cambios.")
    else:
        os.remove(file_path)

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write('''"""
main.py
Este script orquesta la carga del documento, el retriever y el modelo para responder una pregunta fija con contexto.
"""

from loader import load_documents_and_chunk
from retriever import create_or_load_retriever
from generator import ask_with_context

def main():
    # Paso 1: Cargar y dividir el documento
    print("📚 Cargando y dividiendo el documento...")
    chunks = load_documents_and_chunk()

    # Paso 2: Crear o cargar el retriever (FAISS + embeddings)
    print("🔎 Generando o cargando el retriever...")
    retriever = create_or_load_retriever(chunks)

    # Paso 3: Pregunta fija para ejecución automática en Colab
    pregunta = "¿Qué coctel puedo hacer con lima y ron?"
    print("\\n❓ Pregunta:")
    print(pregunta)

    respuesta = ask_with_context(pregunta, retriever)

    # Mostrar respuesta
    print("\\n🧠 Respuesta del modelo:")
    print(respuesta)

if __name__ == "__main__":
    main()
''')
    print(f"✅ Archivo '{file_path}' creado o actualizado con éxito.")



✅ Archivo 'modules/main.py' creado o actualizado con éxito.


### ▶️ Ejecutamos el script `main.py` desde la terminal de Colab

Esta línea ejecuta directamente el script `modules/main.py` usando el sistema de comandos (`!python`) de Google Colab. Al hacerlo:

- Se inicia el flujo completo: carga del PDF, vectorización, generación de respuesta.
- Se lanza automáticamente una pregunta fija (*“¿Qué cóctel puedo hacer con lima y ron?”*).
- Se imprime la respuesta generada por el modelo Flan-T5 usando como contexto el recetario de cócteles.

Este tipo de ejecución es útil para validar que todo el pipeline funciona correctamente desde un único punto.


In [11]:
!python modules/main.py


Device set to use cpu
tokenizer_config.json: 2.54kB [00:00, 8.69MB/s]
spiece.model: 100% 792k/792k [00:00<00:00, 1.37MB/s]
tokenizer.json: 2.42MB [00:00, 31.7MB/s]
special_tokens_map.json: 2.20kB [00:00, 9.10MB/s]
config.json: 1.40kB [00:00, 7.31MB/s]
2025-07-13 10:30:30.711323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752402631.056192    1616 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752402631.156829    1616 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-13 10:30:31.938599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations

### 🤖 Pregunta interactiva al modelo con recuperación contextual (RAG)

En esta celda probamos el sistema completo de pregunta-respuesta. El flujo es:

1. Intentamos cargar el `retriever` guardado en disco.
2. Si no existe, cargamos y dividimos el documento automáticamente.
3. El usuario escribe una pregunta libre.
4. El sistema busca fragmentos relevantes del documento y genera una respuesta basada en contexto.

Esta es la prueba funcional más realista del sistema RAG (retrieval-augmented generation).


In [12]:
import sys
sys.path.append("/content/modules")

from loader import load_documents_and_chunk
from retriever import create_or_load_retriever
from generator import ask_with_context

# 🔍 Intentar cargar retriever desde disco; si no existe, generar los documentos
try:
    retriever = create_or_load_retriever()
except ValueError:
    print("📚 Cargando y dividiendo el documento porque no existe retriever aún...")
    docs = load_documents_and_chunk()
    retriever = create_or_load_retriever(docs)

# 📝 Entrada del usuario
pregunta = input("❓ Escribe tu pregunta: ").strip()

# 🧠 Generar respuesta
respuesta = ask_with_context(pregunta, retriever)

# 📢 Mostrar resultado
print("\n🧠 Respuesta del modelo:\n")
print(respuesta)


Device set to use cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📦 Cargando retriever desde disco...
❓ Escribe tu pregunta: que es mojito


/content/modules/generator.py:26: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(query)



🧠 Respuesta del modelo:

El primero tiene un color claro y un aroma muy peso.


## ✅ Conclusión del ejercicio

Este sistema demuestra con éxito una arquitectura básica de recuperación aumentada por generación (RAG), donde un modelo de lenguaje es asistido por un sistema de recuperación de contexto para responder preguntas basadas en un documento PDF.

Aunque el modelo no siempre ofrece respuestas perfectamente coherentes o precisas —especialmente en recetas compuestas o mal estructuradas dentro del PDF—, **cumple con su objetivo didáctico**: demostrar la conexión funcional entre los módulos de carga, embeddings, búsqueda de contexto y generación de texto.

🧪 En resumen:

- El sistema es **funcional, modular y educativo**.
- Permite experimentar con nuevas preguntas y documentos.
- Sirve como base sólida para mejoras posteriores, como:
  - Filtrado semántico del contexto.
  - Fine-tuning del modelo generador.
  - Añadir validaciones automáticas sobre la respuesta.

⚙️ Además, se ha diseñado **para ejecutarse completamente en local**, lo cual evita el uso de endpoints externos como OpenAI, Google Vertex AI o Anthropic, que pueden requerir:

- API Keys.
- Registros adicionales.
- Costes por token o uso prolongado.

Este enfoque **sin fricciones y sin gastos adicionales** permite explorar conceptos avanzados de IA generativa con total autonomía, facilitando el aprendizaje y la construcción de prototipos reales sin barreras de acceso.

